In [1]:
import pandas as pd
import re

In [2]:
# Users reviews without book titles
reviews_df = pd.read_csv('10000_reviews.csv')
reviews_df.describe()

,Unnamed: 0,book_id,rating,n_votes,n_comments
count,1.000000e+04,1.000000e+04,10000.000000,10000.000000,10000.000000
mean,6.910120e+05,1.337530e+07,3.695800,3.050500,0.939200
std,3.985966e+05,9.243475e+06,1.232321,12.915968,5.122336
min,1.400000e+01,1.000000e+00,0.000000,0.000000,0.000000
25%,3.525825e+05,6.357565e+06,3.000000,0.000000,0.000000
50%,6.938655e+05,1.333094e+07,4.000000,0.000000,0.000000
75%,1.036109e+06,2.035973e+07,5.000000,2.000000,0.000000
max,1.377876e+06,3.625277e+07,5.000000,444.000000,287.000000


In [3]:
# Book titles information

book_names_df = pd.read_csv('book_titles.csv')
book_names_df.describe()

,Unnamed: 0,text_reviews_count,average_rating,num_pages,publication_day,publication_month,publication_year,book_id,ratings_count,work_id
count,6301.000000,6301.000000,6301.000000,5968.000000,5362.000000,5505.000000,5622.000000,6.301000e+03,6.301000e+03,6.301000e+03
mean,3150.000000,2652.332804,3.972349,346.728887,14.029280,6.303724,2011.162220,1.393600e+07,5.144573e+04,2.112064e+07
std,1819.086355,6319.195025,0.274742,144.932054,9.772724,3.274454,5.550981,9.537153e+06,1.955803e+05,1.618190e+07
min,0.000000,20.000000,2.460000,1.000000,1.000000,1.000000,1950.000000,1.000000e+00,4.200000e+01,1.140000e+02
25%,1575.000000,332.000000,3.810000,274.000000,5.000000,4.000000,2010.000000,6.492981e+06,2.561000e+03,6.405906e+06
50%,3150.000000,825.000000,3.990000,336.000000,13.000000,6.000000,2013.000000,1.361296e+07,8.227000e+03,1.884156e+07
75%,4725.000000,2268.000000,4.160000,401.000000,23.000000,9.000000,2014.000000,2.141385e+07,2.861300e+04,3.117587e+07
max,6300.000000,142645.000000,4.770000,1859.000000,31.000000,12.000000,2018.000000,3.625277e+07,4.899965e+06,5.799794e+07


In [4]:
# Merge the two dataset

merged_df = reviews_df.merge(book_names_df, on='book_id', how='left')

In [5]:
merged_df.keys()

Index(['Unnamed: 0_x', 'user_id', 'book_id', 'review_id', 'rating',
       'review_text', 'date_added', 'date_updated', 'read_at', 'started_at',
       'n_votes', 'n_comments', 'Unnamed: 0_y', 'isbn', 'text_reviews_count',
       'series', 'country_code', 'language_code', 'popular_shelves', 'asin',
       'is_ebook', 'average_rating', 'kindle_asin', 'similar_books',
       'description', 'format', 'link', 'authors', 'publisher', 'num_pages',
       'publication_day', 'isbn13', 'publication_month', 'edition_information',
       'publication_year', 'url', 'image_url', 'ratings_count', 'work_id',
       'title', 'title_without_series'],
      dtype='object')

In [6]:
# Check missing values
merged_df.isnull().sum()

Unnamed: 0_x               0
user_id                    0
book_id                    0
review_id                  0
rating                     0
review_text                0
date_added                 0
date_updated               0
read_at                  953
started_at              2920
n_votes                    0
n_comments                 0
Unnamed: 0_y               0
isbn                    1890
text_reviews_count         0
series                     0
country_code               0
language_code            507
popular_shelves            0
asin                    8862
is_ebook                   0
average_rating             0
kindle_asin             1051
similar_books              0
description               61
format                  1271
link                       0
authors                    0
publisher               1383
num_pages                394
publication_day         1372
isbn13                  1641
publication_month       1168
edition_information     8701
publication_ye

In [7]:
# Aggregate reviews for each user
# With rating of 4 and 5 stars

df_high_rated = merged_df[merged_df['rating'] >= 4]

reviews_grouped = (
    df_high_rated
    .groupby('user_id', as_index=False)
    .agg({
        'review_text': lambda x: " ".join(x),
        'book_id': list,
        'rating': list,
        'similar_books': list,
        'authors': list,
        'image_url': list,   # Keeping the thumbnails for displaying book covers
        'title': list

    })
)


reviews_grouped

,user_id,review_text,book_id,rating,similar_books,authors,image_url,title
0,0016a8010771c0c00c97f27dfc5cdd22,One stormy evening Judith McNaught stood at a ...,[129619],[5],"[['107770', '656836', '133499', '330721', '784...","[[{'author_id': '9885', 'role': ''}]]",[https://s.gr-assets.com/assets/nophoto/book/1...,"[A Kingdom of Dreams (Westmoreland, #1)]"
1,00214d8b0a020837cccf5f41eb563037,Even though I gave the book a pretty high rati...,[6582637],[4],"[['6753250', '10557809', '7044446', '188275', ...","[[{'author_id': '71688', 'role': ''}]]",[https://s.gr-assets.com/assets/nophoto/book/1...,"[Archangel's Kiss (Guild Hunter, #2)]"
2,00238d8a4c276c47f5d5e242f54a8f28,Woo the sequel to Hunting Lila! Loved the firs...,"[12410324, 13574417]","[4, 4]","[['13097681', '13181901', '12977172', '1348702...","[[{'author_id': '4573983', 'role': ''}], [{'au...",[https://images.gr-assets.com/books/1334998655...,"[Losing Lila (Lila, #2), Alienated (Alienated,..."
3,002a023d3de233b4bd3ec4fc3e9c581a,Read first time 11/7/15,[13508421],[4],"[['18135513', '15711420', '33283527', '1805398...","[[{'author_id': '835348', 'role': ''}]]",[https://images.gr-assets.com/books/1421837254...,"[Public Enemies (Immortal Game, #2)]"
4,00678bcab8da79ce5720200a3c2e4e7f,"""I think I realized that I would rather die be...",[13206760],[5],"[['9943270', '12812550', '10165761', '15723286...","[[{'author_id': '4684322', 'role': ''}]]",[https://images.gr-assets.com/books/1470056982...,"[Scarlet (The Lunar Chronicles, #2)]"
...,...,...,...,...,...,...,...,...
4324,fff3a250fbc018ad2c2c2d45c86734da,4.5 out of 5 - definitely such fun to read thi...,"[6931344, 6527740]","[5, 5]","[['8673931', '9917925', '8715787', '6933135', ...","[[{'author_id': '3099544', 'role': ''}], [{'au...",[https://images.gr-assets.com/books/1299080471...,"[The Near Witch (The Near Witch, #1), Last Sac..."
4325,fffc34d137f5c5c5e1ca1d6f325a4dcf,There was so much praise and fuss when Dev's d...,"[18938929, 24397040]","[4, 4]","[['25626041', '17900176', '24805314', '2510186...","[[{'author_id': '7025918', 'role': ''}], [{'au...",[https://images.gr-assets.com/books/1450676975...,"[The Bollywood Bride, A Whole New World (A Twi..."
4326,fffce7dae5ac5e8fb6288d81658ececc,** Note: This review is for the first four boo...,[17560419],[4],"[['17830655', '18386330', '35515088', '1849129...","[[{'author_id': '4494819', 'role': ''}]]",[https://images.gr-assets.com/books/1363111811...,"[From the Ashes (Fire and Rain, #1)]"
4327,fffe42f796fb60bd03468ac206c321c1,I can't remember the last time I felt so unsur...,[135479],[4],"[['216377', '4983', '9570', '600349', '968827'...","[[{'author_id': '2778055', 'role': ''}]]",[https://images.gr-assets.com/books/1327867150...,[Cat's Cradle]


In [8]:
# Check to see if there's any duplicated user_id

duplicates_subset = reviews_grouped.duplicated(subset=['user_id'])
duplicates_subset.sum()

np.int64(0)

In [9]:
# Count words in each review
reviews_grouped['word_count'] = reviews_grouped['review_text'].apply(lambda x: len(str(x).split()))

# Filter out reviews less than 15 words

reviews_grouped = reviews_grouped[reviews_grouped['word_count'] >= 15].reset_index(drop=True)

reviews_grouped

,user_id,review_text,book_id,rating,similar_books,authors,image_url,title,word_count
0,0016a8010771c0c00c97f27dfc5cdd22,One stormy evening Judith McNaught stood at a ...,[129619],[5],"[['107770', '656836', '133499', '330721', '784...","[[{'author_id': '9885', 'role': ''}]]",[https://s.gr-assets.com/assets/nophoto/book/1...,"[A Kingdom of Dreams (Westmoreland, #1)]",239
1,00214d8b0a020837cccf5f41eb563037,Even though I gave the book a pretty high rati...,[6582637],[4],"[['6753250', '10557809', '7044446', '188275', ...","[[{'author_id': '71688', 'role': ''}]]",[https://s.gr-assets.com/assets/nophoto/book/1...,"[Archangel's Kiss (Guild Hunter, #2)]",404
2,00238d8a4c276c47f5d5e242f54a8f28,Woo the sequel to Hunting Lila! Loved the firs...,"[12410324, 13574417]","[4, 4]","[['13097681', '13181901', '12977172', '1348702...","[[{'author_id': '4573983', 'role': ''}], [{'au...",[https://images.gr-assets.com/books/1334998655...,"[Losing Lila (Lila, #2), Alienated (Alienated,...",222
3,00678bcab8da79ce5720200a3c2e4e7f,"""I think I realized that I would rather die be...",[13206760],[5],"[['9943270', '12812550', '10165761', '15723286...","[[{'author_id': '4684322', 'role': ''}]]",[https://images.gr-assets.com/books/1470056982...,"[Scarlet (The Lunar Chronicles, #2)]",109
4,006f552534b15a7358a125f7505e0eea,Another excellent novel from Kanae Minato. Bui...,[19161835],[5],"[['7668119', '25367', '7516027', '3190020', '5...","[[{'author_id': '6426380', 'role': ''}, {'auth...",[https://images.gr-assets.com/books/1401076501...,[Confessions],54
...,...,...,...,...,...,...,...,...,...
4010,fff3a250fbc018ad2c2c2d45c86734da,4.5 out of 5 - definitely such fun to read thi...,"[6931344, 6527740]","[5, 5]","[['8673931', '9917925', '8715787', '6933135', ...","[[{'author_id': '3099544', 'role': ''}], [{'au...",[https://images.gr-assets.com/books/1299080471...,"[The Near Witch (The Near Witch, #1), Last Sac...",69
4011,fffc34d137f5c5c5e1ca1d6f325a4dcf,There was so much praise and fuss when Dev's d...,"[18938929, 24397040]","[4, 4]","[['25626041', '17900176', '24805314', '2510186...","[[{'author_id': '7025918', 'role': ''}], [{'au...",[https://images.gr-assets.com/books/1450676975...,"[The Bollywood Bride, A Whole New World (A Twi...",1101
4012,fffce7dae5ac5e8fb6288d81658ececc,** Note: This review is for the first four boo...,[17560419],[4],"[['17830655', '18386330', '35515088', '1849129...","[[{'author_id': '4494819', 'role': ''}]]",[https://images.gr-assets.com/books/1363111811...,"[From the Ashes (Fire and Rain, #1)]",178
4013,fffe42f796fb60bd03468ac206c321c1,I can't remember the last time I felt so unsur...,[135479],[4],"[['216377', '4983', '9570', '600349', '968827'...","[[{'author_id': '2778055', 'role': ''}]]",[https://images.gr-assets.com/books/1327867150...,[Cat's Cradle],41


In [10]:
# Function to clean up the reviews

def clean_text(text):
    text = re.sub(r"<.*?>", "", text)       # Remove HTML tags
    text = re.sub(r"[\r\n]+", " ", text)    # Remove line breaks
    text = re.sub(r"\s+", " ", text)        # Remove space
    text = text.replace(r"\'", "'")         # Replace escaped apostrophes
    return text.strip().lower()

In [11]:
reviews_grouped['cleaned_reviews'] = reviews_grouped['review_text'].apply(clean_text)

In [12]:
# Drop the uncleaned reviews
reviews = reviews_grouped.drop('review_text', axis=1)
reviews.to_csv('processed_reviews.csv', index=False)     # Processed review file

In [13]:
book_names_df.keys()

Index(['Unnamed: 0', 'isbn', 'text_reviews_count', 'series', 'country_code',
       'language_code', 'popular_shelves', 'asin', 'is_ebook',
       'average_rating', 'kindle_asin', 'similar_books', 'description',
       'format', 'link', 'authors', 'publisher', 'num_pages',
       'publication_day', 'isbn13', 'publication_month', 'edition_information',
       'publication_year', 'url', 'image_url', 'book_id', 'ratings_count',
       'work_id', 'title', 'title_without_series'],
      dtype='object')

In [14]:
# Keeping only what we need

book_descriptions_df = book_names_df[[
    'book_id','title','description','average_rating','similar_books','image_url','ratings_count'
]]
book_descriptions_df

,book_id,title,description,average_rating,similar_books,image_url,ratings_count
0,38568,A Quick Bite (Argeneau #1),That hot guy tied to Lissianna Argeneau's bed?...,3.91,"['225669', '50789', '383143', '293616', '75960...",https://images.gr-assets.com/books/1410129015m...,32140
1,16107050,Tied (Fire Born #1),Normal people don't believe their nightmares s...,3.45,"['18074687', '18692520', '25679123', '25744874...",https://images.gr-assets.com/books/1361793669m...,79
2,19057,I Am the Messenger,protect the diamonds\nsurvive the clubs\ndig d...,4.09,"['1121894', '99375', '233906', '824499', '1006...",https://images.gr-assets.com/books/1398483261m...,94968
3,17333426,"Egghead: Or, You Can't Survive on Ideas Alone",A strange and charming collection of hilarious...,4.12,"['15824233', '13152144', '13529076', '18369456...",https://images.gr-assets.com/books/1400879372m...,6461
4,19398490,All the Light We Cannot See,Winner of the Pulitzer Prize\nFrom the highly ...,4.31,"['11807189', '11529868', '17332207', '13593526...",https://images.gr-assets.com/books/1440903901m...,53342
...,...,...,...,...,...,...,...
6296,32766757,No Good Deed,Ellie Hudson is the front-runner on the road t...,3.85,[],https://images.gr-assets.com/books/1481308695m...,295
6297,17061,Coraline,'An electrifyingly creepy tale likely to haunt...,4.03,"['2934112', '24774', '13538708', '34501', '371...",https://images.gr-assets.com/books/1493497435m...,325562
6298,24397043,The Hollow Boy (Lockwood & Co. #3),As a massive outbreak of supernatural Visitors...,4.36,"['24611884', '17464884', '20706799', '18405537...",https://images.gr-assets.com/books/1429031246m...,6088
6299,7715664,Sin Undone (Demonica #5),HER TOUCH IS DEADLY\nAs the only female Seminu...,4.35,"['2318136', '7098304', '9394010', '7023534', '...",https://images.gr-assets.com/books/1344263648m...,23091


In [15]:
# Drop rows with missing description

book_descriptions_df = book_descriptions_df.dropna(subset=['description']).reset_index(drop=True)

In [16]:
# Clean up description

book_descriptions_df['cleaned_description'] = book_descriptions_df['description'].apply(clean_text)
book_descriptions_df.drop('description', axis=1, inplace=True)
book_descriptions_df

,book_id,title,average_rating,similar_books,image_url,ratings_count,cleaned_description
0,38568,A Quick Bite (Argeneau #1),3.91,"['225669', '50789', '383143', '293616', '75960...",https://images.gr-assets.com/books/1410129015m...,32140,that hot guy tied to lissianna argeneau's bed?...
1,16107050,Tied (Fire Born #1),3.45,"['18074687', '18692520', '25679123', '25744874...",https://images.gr-assets.com/books/1361793669m...,79,normal people don't believe their nightmares s...
2,19057,I Am the Messenger,4.09,"['1121894', '99375', '233906', '824499', '1006...",https://images.gr-assets.com/books/1398483261m...,94968,protect the diamonds survive the clubs dig dee...
3,17333426,"Egghead: Or, You Can't Survive on Ideas Alone",4.12,"['15824233', '13152144', '13529076', '18369456...",https://images.gr-assets.com/books/1400879372m...,6461,a strange and charming collection of hilarious...
4,19398490,All the Light We Cannot See,4.31,"['11807189', '11529868', '17332207', '13593526...",https://images.gr-assets.com/books/1440903901m...,53342,winner of the pulitzer prize from the highly a...
...,...,...,...,...,...,...,...
6247,16097364,"Severed Heads, Broken Hearts",3.77,"['16065521', '14290962', '12157780', '15721669...",https://images.gr-assets.com/books/1374501795m...,1119,"robyn schneider's book, originally titled seve..."
6248,32766757,No Good Deed,3.85,[],https://images.gr-assets.com/books/1481308695m...,295,ellie hudson is the front-runner on the road t...
6249,17061,Coraline,4.03,"['2934112', '24774', '13538708', '34501', '371...",https://images.gr-assets.com/books/1493497435m...,325562,'an electrifyingly creepy tale likely to haunt...
6250,24397043,The Hollow Boy (Lockwood & Co. #3),4.36,"['24611884', '17464884', '20706799', '18405537...",https://images.gr-assets.com/books/1429031246m...,6088,as a massive outbreak of supernatural visitors...


In [17]:
book_descriptions_df.to_csv('book_descriptions.csv', index=False)